In [1]:
import sys,os
#import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch.utils
#import cv2

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from random import randint

In [4]:
device = torch.device("cuda")
#device = torch.device("cpu")
print(device)

cuda


In [5]:
path = 'C:/Users/vigne/Desktop/age_gender_folder'


In [6]:
x_train = np.load(path+'/x_train2.npy')
y_train = np.load(path+'/y_train2.npy')
x_val = np.load(path+'/x_val2.npy')
y_val = np.load(path+'/y_val2.npy')

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [7]:
print(y_train.shape)

(21334, 2)


In [8]:
y_train_age = y_train[:,0]
print(y_train_age.shape)

(21334,)


In [10]:
y_val_age = y_val[:,0]

In [12]:
#size = 224
transform = transforms.Compose(
    [transforms.ToPILImage(),
     #transforms.Resize((size,size)),
     transforms.RandomRotation(degrees = 20),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))
])

class MyDataset(Dataset):
    def __init__(self, data, target, transform=transform):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).float().unsqueeze(1)
        self.transform = transform
     
    def __len__(self):
        return len(self.target)
    
    def __getitem__(self, index):
        
        return self.data[index], self.target[index]
        

In [14]:
class myAgeModel(nn.Module):
    def __init__(self):
        super(myAgeModel , self).__init__()
     
        self.conv1 = nn.Conv2d(1,  16,  kernel_size=3 )
        self.pool1  = nn.MaxPool2d(2,2)
        self.batch1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16,  32, kernel_size=3 )
        self.pool2  = nn.MaxPool2d(2,2)
        self.batch2 = nn.BatchNorm2d(32)
      
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3 )
        self.pool3  = nn.MaxPool2d(2,2)
        self.batch3 = nn.BatchNorm2d(64)
    
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3 )
        self.pool4  = nn.MaxPool2d(2,2)
        self.batch4 = nn.BatchNorm2d(128)

        self.linear1 = nn.Linear(128*1*1, 64)
        self.linear2 = nn.Linear(64,32)
        self.linear3 = nn.Linear(32,1)
        self.dropout1 = nn.Dropout(p=0.2)
        self.dropout2 = nn.Dropout(p=0.4)
        
    
    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.batch1(x)
        x = self.pool1(x)
    
        x = self.conv2(x)
        x = F.relu(x)
        x = self.batch2(x)
        x = self.pool2(x)
    
        x = self.conv3(x)
        x = F.relu(x)
        x = self.batch3(x)
        x = self.pool3(x)
    
        x = self.conv4(x)
        x = F.relu(x)
        x = self.batch4(x)
        x = self.pool4(x)

        x = x.view(-1, 128)

        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        
        return x

In [13]:
batchsize = 100
in_channel = 1
num_classes = 16
learning_rate = 0.01
num_epochs = 30

In [15]:
train_dataset = MyDataset(x_train, y_train_age)
train_loader = DataLoader(dataset=train_dataset, batch_size=batchsize, shuffle=True)

val_dataset = MyDataset(x_val, y_val_age)
val_loader = DataLoader(dataset=val_dataset, batch_size=batchsize, shuffle=True)

In [16]:
len(train_loader)

214

In [17]:
def eval_on_val_set(val_acc,val_loss,val_loader,val_dataset):

    running_loss=0
    running_acc=0
    num_batches=0
    model.eval()
    for (data, targets) in val_loader:

        data=data.to(device)
        targets=targets.to(device)

        scores = model(data) 

        loss = criterion(scores, targets)

        running_loss += loss.detach().item()*data.size(0)
        _, predictions = torch.max(scores.data, 1)
        running_acc += (predictions == targets).sum().item()

        num_batches += 1

    lo =  running_loss/len(val_dataset)
    acc = 100*running_acc/len(val_dataset)
    
    val_acc.append(acc)
    val_loss.append(loss)
    
    print('val loss = {}  val accuracy = {}'.format(lo,acc))
    
    return val_acc,val_loss

In [23]:
def eval_on_val_set2(val_acc,val_loss,val_loader,val_dataset):

    running_loss=0
    running_acc=0
    num_batches=0
    model.eval()
    for (data, targets) in val_loader:

        data=data.to(device)
        targets=targets.to(device)

        scores = model(data) 

        loss = criterion(scores, targets)

        running_loss += loss.detach().item()*data.size(0)
        _, predictions = torch.max(scores.data, 1)
        running_acc += (predictions == targets).sum().item()

        num_batches += 1
        break

    #lo =  running_loss/len(val_dataset)
    #acc = 100*running_acc/len(val_dataset)
    
    #val_acc.append(acc)
    #val_loss.append(loss)
    
    #print('val loss = {}  val accuracy = {}'.format(lo,acc))
    
    #return val_acc,val_loss
    return scores,targets

In [18]:
model = myAgeModel()

In [19]:
model = model.to(device)

In [20]:
criterion = nn.MSELoss()


In [21]:
losses=[]
accuracy=[]

val_loss = []
val_acc = []

for epoch in range(40):
    model.train()
    running_loss=0
    running_acc=0
    num_batches=0
    
    # divide the learning rate by 2 at epoch 10, 14 and 18
    if epoch%4==0 and epoch!=0:
        learning_rate = learning_rate / 2

   # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.RMSprop( model.parameters() , lr=learning_rate )
    
    for count, (data, targets) in enumerate(train_loader):
        num_batches += 1
        optimizer.zero_grad()
        data.requires_grad_()
        
        data = data.to(device)
        targets = targets.to(device)
        
        scores = model(data)
        
        loss = criterion(scores, targets)
        running_loss += loss.detach().item()*data.size(0)
        
        loss.backward()
        optimizer.step()
        
        _, predictions = scores.max(1)
        running_acc += (predictions == targets).float().sum()
    
    lo = running_loss/len(train_dataset)
    acc = 100*running_acc/len(train_dataset)
    losses.append(lo)
    accuracy.append(acc)
    print('Epoch {} : loss = {}  accuracy = {}'.format(epoch,lo,acc))
    
    #if epoch%5==0:
    #    val_acc,val_loss = eval_on_val_set(val_acc,val_loss,val_loader,val_dataset)

Epoch 0 : loss = 454.50401896649026  accuracy = 0.0
Epoch 1 : loss = 428.56024284502263  accuracy = 0.0
Epoch 2 : loss = 443.90842333963775  accuracy = 0.0
Epoch 3 : loss = 292.1005485222111  accuracy = 0.0
Epoch 4 : loss = 194.3032381281786  accuracy = 0.0
Epoch 5 : loss = 192.27664485826614  accuracy = 0.0
Epoch 6 : loss = 173.47584162342233  accuracy = 0.0
Epoch 7 : loss = 164.7087404124681  accuracy = 0.0
Epoch 8 : loss = 139.89829343769134  accuracy = 0.0
Epoch 9 : loss = 130.5506644666927  accuracy = 0.0
Epoch 10 : loss = 126.65081208986228  accuracy = 0.0
Epoch 11 : loss = 126.21487842165364  accuracy = 0.0
Epoch 12 : loss = 114.43286740832401  accuracy = 0.0
Epoch 13 : loss = 109.73752123799713  accuracy = 0.0
Epoch 14 : loss = 107.00159313158633  accuracy = 0.0
Epoch 15 : loss = 104.796107448722  accuracy = 0.0
Epoch 16 : loss = 98.61542458380913  accuracy = 0.0
Epoch 17 : loss = 98.22908305469176  accuracy = 0.0
Epoch 18 : loss = 96.66374542559613  accuracy = 0.0
Epoch 19 : l

In [ ]:
plt.plot(losses,'-o')
plt.title('Loss per epoch for 30 epochs')
plt.ylabel('Loss')
plt.xlabel('No. of epochs')

In [27]:
scorestest,targetstest = eval_on_val_set2(val_acc,val_loss,val_loader,val_dataset)

In [35]:
scorestest.T.round()

tensor([[31., 27., 20., 31.,  6., 51.,  3., 13., 21., 39., 82., 26., 30.,  3.,
         25., 23., 50., 21., 43., 17., 22., 26., 43., 60., 34., 75., 19.,  5.,
         29., 27., 23., 26., 25., 26.,  7., 43., 52., 59., 25., 29., 29., 21.,
         34., 35.,  3., 25., 40., 20.,  4., 56., 39., 29., 25., 35.,  9., 25.,
         26., 38., 47., 33., 21., 30., 20., 32., 16., 33., 28., 75., 31., 26.,
         46., 23., 44.,  4., 28., 27., 25., 10., 23., 22., 23., 63., 59., 25.,
         26., 15., 78., 63., 24.,  6., 55., 21., 24., 25., 30., 34., 12., 58.,
         36., 66.]], device='cuda:0', grad_fn=<RoundBackward>)

In [33]:
targetstest.T

tensor([[ 35.,  27.,  18.,  36.,   6.,  63.,   1.,   9.,  30.,  49., 100.,  25.,
          35.,   1.,  24.,  22.,  45.,  18.,  61.,  15.,  24.,  33.,  35.,  60.,
          26.,  78.,  12.,   1.,  23.,  18.,  25.,  35.,  23.,  35.,  14.,  54.,
          45.,  61.,  28.,  26.,  34.,  28.,  40.,  32.,   1.,  30.,  49.,  25.,
           1.,  60.,  45.,  36.,  24.,  40.,  13.,  28.,  18.,  52.,  34.,  36.,
          27.,  34.,  26.,  33.,   8.,  34.,  24.,  75.,  26.,  24.,  36.,  63.,
          26.,   4.,  35.,  22.,  21.,   8.,  26.,  23.,  30.,  66.,  80.,  16.,
          20.,  23.,  75.,  55.,  28.,   9.,  59.,  22.,  26.,  40.,  25.,  52.,
           9.,  60.,  29.,  67.]], device='cuda:0')

In [39]:
print(scorestest[35])
print(targetstest[35])

tensor([43.0248], device='cuda:0', grad_fn=<SelectBackward>)
tensor([54.], device='cuda:0')


In [ ]:
import pickle as pk
dest_path = "/content/drive/My Drive/cv_project/model.pkl"
#model = myModel()
pk.dump(model.to('cpu'),open(dest_path,'wb'))